Public Certificate     :subject.crt
Private Key in         :cert_bckup.p12
Password on .p12 file  :CSE539_Rocks!
CA Public Certificate  :root.cer

#Done
1. Verify Subject's certificate: True if valid, False otherwise
Throws exception- use Try Catch

#Done
2. From Subject's Certificate, print:
subject name, issuer, serial number,
encryption algorithm, not valid before, not valid after

#Done
3. Print subject's public & private key.
the public key is represented by the integers n and e
the private key, by the integer d
    a. Public Key Modulus (n)
    b. Public Key Exponent (e)
    c. Private Key Exponent (d)

#Done
4. Print public key of CA:
    a. Root Public Key Modulus (n)
    b. Root Public Key Exponent (e)

#Done
5. Print hex signature on subject's certificate

#Done
6. Encrypt string using RSA:
    b'Hello World'
    
    Use OEAP padding,
    Mask generation function: MGF1
        & SHA256 hash function
    Encrypt & Decrypt to verify!
    
    Encrypt using subject's public key,
    Decrypt using subject's private key.

Input arguments:
    p12 file path
    CA (root) crt file path
    client (subject) crt file path
    password

In [1]:
#InputArguments
p12FilePath="/Users/atinsinghal97/Desktop/Project 4/Certificates/cert_bckup.p12" #sys.argv[1]
rootCertificatePath="/Users/atinsinghal97/Desktop/Project 4/Certificates/root.crt" #sys.argv[2]
clientCertificatePath="/Users/atinsinghal97/Desktop/Project 4/Certificates/subject.crt" #sys.argv[3]
password="CSE539_Rocks!" #sys.argv[4]

#check len(sys.argv)

In [2]:
import OpenSSL.crypto
import cryptography.x509
from cryptography.hazmat.primitives.serialization import load_pem_public_key, load_pem_private_key
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.backends import default_backend
#from cryptography.hazmat.primitives import hashes

In [3]:
#Initializing certificate objects

certObject=OpenSSL.crypto

clientCertificateFile=open(clientCertificatePath, 'rt').read()
clientCertificate=certObject.load_certificate(certObject.FILETYPE_PEM, clientCertificateFile)

p12File=open(p12FilePath, 'rt').read()
p12=OpenSSL.crypto.load_pkcs12(p12File, password)

rootCertificateFile=open(rootCertificatePath, 'rt').read()
rootCertificate=certObject.load_certificate(certObject.FILETYPE_PEM, rootCertificateFile)


In [4]:
#1- Verify Subject's certificate: True if valid, False otherwise

try:
    #print('Verify: ', clientCertificate.verify(clientCertificate.get_pubkey()))
    storeObject=OpenSSL.crypto.X509Store()
    storeObject.add_cert(rootCertificate)
    #print (storeObject)

    #forVerification=OpenSSL.crypto.X509StoreContext
    #forVerification.set_store(storeObject)
    forVerification = OpenSSL.crypto.X509StoreContext(storeObject, clientCertificate)
    forVerification.set_store(storeObject)
    forVerification.verify_certificate()
    print ('Certificate Verification: True')

except Exception as e:
    print('Certification Verification: False')
    #print('Error: ', e)


Certification Verification: False


In [14]:
#2- Print Subject Name, Issuer, Serial Number, Encryption Algorithm, Not Valid Before, Not Valid After

print('Subject Common Name: ', clientCertificate.get_subject().commonName)
print('Issuer Common Name: ', clientCertificate.get_issuer().commonName)
print('Serial Number: ', clientCertificate.get_serial_number())
print('Encryption Algorithm:', clientCertificate.get_signature_algorithm())
print('Not Valid Before: ', clientCertificate.get_notBefore())
print('Not Valid After: ', clientCertificate.get_notAfter())

('Subject Common Name: ', u'sundevilsafe.com')
('Issuer Common Name: ', u'Sectigo RSA Domain Validation Secure Server CA')
('Serial Number: ', 279175654248908418197316606967672119883L)
('Encryption Algorithm:', 'sha256WithRSAEncryption')
('Not Valid Before: ', '20190322000000Z')
('Not Valid After: ', '20190620235959Z')


In [12]:
#3- Print Subject's Public & Private Key

#print(clientCertificate.get_pubkey())
pubKeyObject=clientCertificate.get_pubkey()
pubKeyString=certObject.dump_publickey(certObject.FILETYPE_PEM, pubKeyObject)
print ('Public Key of Subject(n): ')
#print (pubKeyString)

pubKey = load_pem_public_key(pubKeyString, backend=default_backend()).public_numbers().n
print (pubKey)
e = load_pem_public_key(pubKeyString, backend=default_backend()).public_numbers().e
print ('e: ', e)

p12Certificate=p12.get_certificate()
p12Key=p12.get_privatekey()
#print (p12Key)
privateKeyString=OpenSSL.crypto.dump_privatekey(certObject.FILETYPE_PEM, p12Key)
print ('Private Key of Subject(d): ')
#print (privateKeyString)

privateKey = load_pem_private_key(privateKeyString, password=None,backend=default_backend()).private_numbers().d
print (privateKey)

Public Key of Subject(n): 
21077153818761509013420887295789777526381470891059241603835731174578561528272514732364372022128313222798429799591634889643647267740984746901232961644484807420124065572225124454192930109284610798617935357525847868490316254559361469173074997262741836934984543216611426323596674966026821843549785698420735419753126471392900277945728904520317719259592910223328604555566083744041993316206931791473030770736422668152835623336582614954598264269116246863755011903025431632958306918948555150293497429964013313496981504332051411997440129210325623096165343650017640049684435715502081065400604868571467789815798919466984120998231
('e: ', 65537)
Private Key of Subject(d): 
204693167081604858961685645317169228103453236767512634139391913280803470648635235847047497763461975935952463715856506373674293551720471822943188475179902122139957032130767733667396649627788519996282364420166873565436800380539188462770450943860983825228643548241095480601485812832859168084656485702333727094772229

In [7]:
#4- Print Public key of CA

#print(rootCertificate.get_pubkey())
rootPubKeyObject=rootCertificate.get_pubkey()
rootPubKeyString=certObject.dump_publickey(certObject.FILETYPE_PEM, rootPubKeyObject)
print ('Public Key of Certification Authority(n): ')
#print (rootPubKeyObject)
#print (rootPubKeyString)

rootPubKey = load_pem_public_key(rootPubKeyString, backend=default_backend()).public_numbers().n
print (rootPubKey)
eRoot = load_pem_public_key(rootPubKeyString, backend=default_backend()).public_numbers().e
print ('e: ', eRoot)


Public Key of Certification Authority(n): 
27071805731268068429094433253870941103630115656544083986688557250096650002741356969068276793574297189996744882329828714710418422450500008985954757761749612179194262954838266897268532461471487935006536720272494786369295065485985775416665556679189441271529970596944264659076045397506534017596880375081936227230390079744485923579156075147392924215968126808432802589158460341539735535162275393422353916283261069707744210608603163551594107036190302366892832801673071921401850194237815007641724075220215236537097134132878656895768334289537403738520740616524362273323209296475725089291190047065303870231628551068954108124949
('e: ', 65537)


In [8]:
#5- Print Hex Signature on Subject's Certificate

x509Object=cryptography.x509.Certificate

x509Certificate=cryptography.x509.load_pem_x509_certificate(clientCertificateFile, default_backend())
#x509Certificate=x509Object.load_certificate(certObject.FILETYPE_PEM, clientCertificateFile)

print ("Hex Signature of Subject's Certificate: ")
print (x509Certificate.signature.encode('hex'))

Hex Signature of Subject's Certificate: 
2507d45cba6f3cf90bb9def0bd750413b3cd6041875be309d016846dda8ff56e026929d3cabc8021fcc58983201d5674ac08baaafc8fad36176523e40cf68f7a6ccb1829bdcdf243c161f352b1063a69c497254d388d6da96680b36eb1bef61e1f92e0647ff33fc3ce1fc1b96ea6fb8feea16c019d2c7f01eeef999b15c5b7ac6faf5ba266023aa5a7cfde17385c4f8f641f0690dda072ca283391afb6d570bca1e786ce7f981d412da48fd7666a5a902981ce39f406d1092bfd54c046e2a6cb4933075b9d3c5bd590ad2f84112e95ab8a7e581bd4860ae38813316874ec67b532cf137720694c00ba539aba66775a5e7be5e575ebe168415b0a1f12b5ddc85a


In [9]:
#6- Encrypt String using RSA: b'Hello World'

message = b'Hello World'

#publicKey serialization
serializedPubKey= serialization.load_pem_public_key(
    pubKeyString,
    backend=default_backend()
)

#encryption
cipherText= serializedPubKey.encrypt(
    message,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

#privateKey serialization
serializedPrivateKey= serialization.load_pem_private_key(
    privateKeyString,
    password= None,
    backend=default_backend()
)

plainText= serializedPrivateKey.decrypt(
    cipherText,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

print ('CipherText: ', cipherText.encode('hex'))
print ('PlainText: ', plainText)


('CipherText: ', '06de04166ddace3b89e4520405a3c9ae03e2840d606eb362ffc4fbf395c61f7f2aa25ae5ffa06b9219e158ad4170aaf84c7b41eed51172e2a5217c8fa557abeca4599f5d69aae3563c4141ff3dae5ce2c835b9729d03233c9e516de21d75f14ef688eba99a57673781abbd2eda763cfe8356ef0b2ce6a022d606056d5998382aa5a326a096d683a5c61e376c5b4015cd3a0e700a2461c0fcab54be3858c20d46da4369e2e5634e03459a2873d9de5446aaace4d102045e598d0ecace2d04fdb53f89387277756825ff28bae22dcce75e2ae80143f75ebb3492753055cf9831327c265a8a3194a4792b5468ad065d563f749d60712c2c7264ac312de6bf29c853')
('PlainText: ', 'Hello World')
